In [1]:
import pickle
import cvxopt
import numpy as np
from scipy.sparse.csc import csc_matrix

/tmp/ipykernel_652/3996406306.py:4: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  from scipy.sparse.csc import csc_matrix


### Get Quadratic Problem

In [2]:
EXAMPLE_NUM = 6

In [3]:
with open(f"Data/problem{EXAMPLE_NUM}.pickle", "rb") as f:
    E_sparse, Q_diag, q, b, u, solution = pickle.load(f)
Q = np.diag(Q_diag)
E = E_sparse.toarray()
E, E_sparse, b = E[:-1], E_sparse[:-1], b[:-1]              # linear independence
u = np.nan_to_num(u, posinf=1e6)   # np.inf doesn't work with cvxopt

In [4]:
E, Q_diag, q, b, u, solution

(array([[-1., -1., -1., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., -1.,  0.,  0.]]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.]),
 array([1.e+05, 1.e+05, 1.e+05, 3.e+00, 1.e+05, 1.e+05, 1.e+05, 1.e+05,
        1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05,
        1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 2.e+00,
        1

In [5]:
n, m = E.shape
print(f"#nodes:{n+1}, #edges:{m}")

#nodes:20, #edges:108


### Init and solve with cvxopt

In [6]:
P_cvxopt = cvxopt.matrix(2*Q)
p_cvxopt = cvxopt.matrix(q)
# x >= 0  <>  G = -I,  h = 0
# x <= u  <>  G =  I,  h = u
G_cvxopt = cvxopt.matrix(np.block([
    [-np.eye(m)], 
    [np.eye(m)]
]))
h_cvxopt = cvxopt.matrix(np.concatenate((np.zeros(m), u)))
# Ex = b
A_cvxopt = cvxopt.matrix(E)
b_cvxopt = cvxopt.matrix(b)

sol = cvxopt.solvers.qp(P_cvxopt, p_cvxopt, # cost
                        G_cvxopt, h_cvxopt, # ineq. constraints
                        A_cvxopt, b_cvxopt) # eq. constraints

     pcost       dcost       gap    pres   dres
 0: -1.5147e+11 -5.0499e+10  3e+11  1e+04  7e-17
 1: -1.0932e+10 -3.5468e+08  6e+10  2e+03  4e-13
 2: -1.3527e+08 -1.1941e+07  7e+08  2e+01  5e-15
 3: -1.8355e+06 -1.0783e+07  3e+07  6e-01  3e-16
 4:  3.4071e+05 -5.1951e+06  1e+07  2e-01  2e-16
 5:  4.1045e+05 -1.7620e+06  4e+06  6e-02  2e-16
 6:  8.7267e+04 -4.0324e+05  8e+05  1e-02  2e-16
 7:  3.5688e+04 -9.4175e+04  2e+05  2e-03  3e-16
 8:  8.7926e+02 -6.7148e+02  2e+03  3e-05  2e-16
 9:  4.4934e+02  4.3383e+02  2e+01  3e-07  2e-16
10:  4.4504e+02  4.4489e+02  2e-01  3e-09  2e-16
11:  4.4500e+02  4.4500e+02  2e-03  3e-11  2e-16
12:  4.4500e+02  4.4500e+02  2e-05  3e-13  2e-16
Optimal solution found.


In [7]:
1/2* solution @ Q @ solution + solution @ q

295.0

In [8]:
assert(sol["status"] == 'optimal')
sol_cvxopt = np.array(sol["x"]).reshape(m)
1/2* sol_cvxopt @ Q @ sol_cvxopt + sol_cvxopt @ q

295.0000043439088

In [9]:
np.max(sol_cvxopt - solution)

6.838085653271264e-12

In [10]:
sol_cvxopt

array([ 7.04416129e-13,  1.89767903e-12,  6.28306960e-13,  5.00000000e+00,
        6.78531344e-13,  7.50319183e-13,  5.55716595e-13,  9.36598939e-13,
        5.91591857e-13,  3.21342959e-13,  4.14809573e-13,  4.54016828e-13,
        8.61929097e-13,  3.45078210e-13, -3.94647638e-14,  1.92126868e-13,
        6.28126458e-13,  2.14968239e-13,  1.14901251e-13, -3.98969739e-13,
        8.31375507e-14,  1.19807787e-14,  3.11195716e-13,  5.00000000e+00,
        1.15026076e-13,  6.68839250e-14, -1.59208084e-14,  3.63658342e-13,
        2.09346948e-13,  2.91538821e-13,  6.65372366e-14,  2.50426144e-13,
        5.66198364e-13,  1.42458078e-13,  1.56123664e-13,  9.11949777e-14,
        1.63761709e-13,  8.17113704e-14,  5.00000000e+00,  1.24969961e-12,
        3.95746197e-13,  1.30028244e-13,  7.62445445e-14,  2.26205567e-13,
        6.04432682e-13,  9.53502155e-14,  3.03757673e-14,  1.59664284e-12,
        3.41094594e-13,  4.47378796e-13,  5.42207639e-14,  2.37546334e-13,
        5.00000000e+00,  

In [11]:
np.argwhere(sol_cvxopt > 1e-6).flatten()

array([  3,  23,  38,  52,  58,  69,  75,  80,  82,  87,  92, 105])